# F&B데이터 전처리 


1. VISITJEJU 로그 데이터 전처리
    - 현재 운영되고 있는 식당 및 카페 정보만 반영하기 위해 VISITJEJU 크롤링 데이터로 필터링 
    

2. 주소 변환 및 좌표 추출  
    - 카카오API 활용해 도로명주소->지번주소 변환 및 좌표 추출


3. 매출액 상위 가맹점 데이터로 가격정보 추가  
    - 제주지역 매출액상위 가맹점 데이터(JEJU_MERCHANT)의 매출구간 중위수 /20대 고객의 요일시간대별 승인건수합으로 대체
    - NA값은 추후 네이버지도 크롤링 데이터의 메뉴평균 가격정보 대체 


4. 지번주소 기준 온실가스배출량 데이터 병합 
    - **공간 탄소배출량 계산**
   

5. 가격정보 대체
    - 네이버맵 크롤링 데이터 정제
    - 네이버맵 크롤링 데이터 내 메뉴 평균 데이터로 가격 결측 데이터 대체  


6. 메뉴군 분류 및 평균 탄소 배출량 NA대체
    - 중분류 카테고리 생성하고 카테고리 평균 탄소배출량으로 결측치 대체
    - 카테고리 **음식별 탄소배출량**도 고려하여 계산


In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
import numpy as np
import re
import requests
import json
import matplotlib.pyplot as plt
import collections
import ast

In [3]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

## 1. VISITJEJU 로그 데이터 전처리
- 비짓제주 로그 데이터 중 음식점 데이터만 추출
- 크롤링 데이터 기준 필터링하여 현재 운영 중인 정보만 반영

### 1-(1) 음식점 데이터 필터링

In [4]:
vj_view = pd.read_csv('data/PRE_JT_MT_ACCTO_TRRSRT_SCCNT_LIST.csv')

In [5]:
vj_view.head(6)[['AREA_NM_PRE', 'CL_NM', 'MAR_VIEW_CO', 'NOV_VIEW_CO']]

,AREA_NM_PRE,CL_NM,MAR_VIEW_CO,NOV_VIEW_CO
0,1080칼국수,음식점,9,69
1,1100고지,관광지,438,541
2,1100고지습지,관광지,613,960
3,1112도로,관광지,60,112
4,1915지오하우스,숙박,38,47
5,1950하우스,음식점,17,48


In [6]:
# 음식점 데이터만 추출
vj_fb = vj_view[vj_view.CL_NM == "음식점"]
vj_fb = vj_fb.reset_index(drop=True) ; vj_fb

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO
0,1080칼국수,음식점,['1080칼국수'],['제주특별자치도 제주시 서광로13길 3'],12129,9,14,15,18,23,69
1,1950하우스,음식점,['1950하우스'],['제주특별자치도 제주시 애월읍 평화로 2369'],12129,17,25,26,21,27,48
2,2093하우스,음식점,['2093하우스'],['제주특별자치도 제주시 조천읍 남조로 2093'],12129,13,21,26,19,35,55
3,24시누름돌김치찌개,음식점,['24시누름돌김치찌개'],['제주특별자치도 제주시 노연로 146'],4044,0,6,1,0,0,0
4,24시뼈다귀탕동홍점,음식점,['24시뼈다귀탕동홍점'],['제주특별자치도 서귀포시 동홍남로 43'],4044,0,3,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1734,흑돼지촌,음식점,['흑돼지촌'],['제주특별자치도 제주시 한림읍 한림상로 62'],12129,11,18,14,34,29,37
1735,흑돼지킹생구이전문점,음식점,['흑돼지킹생구이전문점'],['제주특별자치도 제주시 도령로11길 26-9'],12129,15,18,16,11,18,21
1736,흑소랑,음식점,['흑소랑'],['제주특별자치도 제주시 연북로 631'],12129,18,30,33,48,54,57
1737,흑염소마루,음식점,['흑염소마루'],['제주특별자치도 서귀포시 남원읍 태위로663번길 1-1'],12129,4,14,30,27,25,36


In [7]:
# VISIT JEJU 크롤링 데이터
vj_craw_fb = pd.read_csv('data/VISITJEJU_음식점정보크롤링_0915.csv')

In [8]:
#안 쓸 열들 drop
vj_craw_fb = vj_craw_fb.drop(['stars', 'address', 'telephone', 'numreview', 'view', 'rec_tour', 'rec_restaurant', 'rec_lodgment', 'rec_bigdata', 'detail_side_box'], axis=1)

### 1-(2) 이름 전처리

In [9]:
def name_preprocessing(name):
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = name.split("(")[0]
    name = re.sub('[\[.\]]','',name)
    return name.strip()

In [10]:
# 로그 데이터 이름 preprocessing
vj_fb['AREA_NM_PRE'] = list(map(lambda x : name_preprocessing(x), vj_fb['AREA_NM_PRE']))
#크롤링 데이터 이름 preprocessing
vj_craw_fb['name_PRE'] = list(map(lambda x : name_preprocessing(x), vj_craw_fb['name']))

### 1-(3) 중복행 삭제

In [11]:
# 중복행 확인
dup_name = vj_craw_fb['name_PRE'].value_counts().to_frame()
dup_name[dup_name.name_PRE == 2]

,name_PRE
커피가게쉬고가게,2
제니의정원,2
일도국수,2
어진이네횟집,2
24시전주명가콩나물국밥,2
바다를본돼지,2


In [12]:
#vj_craw_fb[vj_craw_fb.name_PRE == "24시전주명가콩나물국밥"]  
#vj_craw_fb[vj_craw_fb.name_PRE == "제니의정원"]              
#vj_craw_fb[vj_craw_fb.name_PRE == "일도국수"]
#vj_craw_fb[vj_craw_fb.name_PRE == "바다를본돼지"]
#vj_craw_fb[vj_craw_fb.name_PRE == "어진이네횟집"]

In [13]:
# 중복행 삭제 
vj_craw_fb = vj_craw_fb.drop([321,608,940,1192,1325,1476], axis=0).reset_index(drop=True)

### 1-(4) 데이터 병합

In [14]:
vj_filter_fb = pd.merge(left=vj_fb, right=vj_craw_fb, how='inner',
                    left_on='AREA_NM_PRE', right_on='name_PRE').reset_index(drop=True)

In [15]:
final_fb = vj_filter_fb.drop(['CL_NM','AREA_NM','name','name_PRE'], axis=1)

## 2. 주소 변환 및 좌표 추출
- 카카오API 이용해서 도로명주소->지번주소 변환, 좌표추출

### 2 - (1) 주소 변환

In [16]:
# 주소 list 문자열로 변환
final_fb['ADDR'] = [ast.literal_eval(x)[0] for x in final_fb['ADDR']]

In [17]:
# kakao api key 
api_key = '7465f66e54fbe4f71543e474623685f1'

In [18]:
addrs = final_fb['ADDR']
locations_add = []
locations_road = []
miss_index = []

# 도로명주소, 지번주소 추출 
for i in range(len(addrs)):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addrs.iloc[i])
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers = headers).text))
    
    try:
        addr_temp = result['documents'][0]['address']['address_name']
        road_temp = result['documents'][0]['road_address']['address_name']
        locations_add.append(addr_temp)
        locations_road.append(road_temp)

    except TypeError:
        print(i, end=' ')
        miss_index.append(i)
        locations_add.append(np.NaN)
        locations_road.append(np.NaN)

    except IndexError:
        print(i, end=' ')
        miss_index.append(i)
        locations_add.append(np.NaN)
        locations_road.append(np.NaN)

69 92 180 419 496 639 646 667 682 737 740 820 937 1051 1102 1201 1291 1309 1310 1434 1447 1518 

In [19]:
# 검색된 주소 변수 생성
final_fb['LN_ADDR'] = locations_add
final_fb['ROAD_ADDR'] = locations_road

### 2 - (2) 주소 검색 안되는 데이터 정제 
- 주소 오류 있는 경우  삭제

In [20]:
#final_fb.iloc[miss_index][['AREA_NM_PRE','ADDR']]

In [21]:
final_fb = final_fb.dropna().reset_index(drop=True)

### 2 - (3) 좌표 추출

In [22]:
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

In [23]:
lat_list = []
lon_list = []

# 좌표 추출 함수 
for i in range(len(final_fb)):
    try:
        lat_list.append(addr_to_lat_lon(final_fb['ADDR'][i])[1])
        lon_list.append(addr_to_lat_lon(final_fb['ADDR'][i])[0])
    except TypeError:
        print(i, end = " ")
        lat_list.append(np.NaN)
        lon_list.append(np.NaN)
    except IndexError:
        print(i, end=' ')
        lat_list.append(np.NaN)
        lon_list.append(np.NaN)
    except KeyError:
        print(i, end=" ")
        lat_list.append(np.NaN)
        lon_list.append(np.NaN)

In [24]:
# 좌표 변수 생성
final_fb['lon'] = lon_list
final_fb['lat'] = lat_list

### 2 - (4) 읍면동 변수 추출

In [25]:
# 제주시 뒤에 나오는 단어 추출
final_fb['region'] = [tmp.split('시')[1].strip().split(' ')[0] for tmp in final_fb['LN_ADDR']]

### 2 - (5) 주소 형식 통일 
- 공백 제거
- 제주 -> 제주특별자치도로 변경

In [26]:
def addr_preprocessing(addr):
    addr = addr.strip()
    if '제주특별자치도' in addr:
        addr = "제주특별자치도 " + addr.split('제주특별자치도')[1].lstrip()
    else:
        addr = re.sub('제주 서귀포시','제주특별자치도 서귀포시', addr)
        addr = re.sub('제주 제주시','제주특별자치도 제주시', addr)

    addr = re.sub("  ", " ", addr)
    return addr.strip() 

In [27]:
final_fb['ROAD_ADDR'] = list(map(lambda x : addr_preprocessing(x), final_fb['ROAD_ADDR']))
final_fb['LN_ADDR'] = list(map(lambda x: addr_preprocessing(x), final_fb['LN_ADDR']))

In [28]:
final_fb[['ROAD_ADDR', 'LN_ADDR']]

,ROAD_ADDR,LN_ADDR
0,제주특별자치도 제주시 서광로13길 3,제주특별자치도 제주시 삼도일동 518-9
1,제주특별자치도 제주시 애월읍 평화로 2369,제주특별자치도 제주시 애월읍 유수암리 1023
2,제주특별자치도 제주시 조천읍 남조로 2093,제주특별자치도 제주시 조천읍 교래리 786-21
3,제주특별자치도 제주시 노연로 146,제주특별자치도 제주시 연동 304-3
4,제주특별자치도 서귀포시 동홍남로 43,제주특별자치도 서귀포시 동홍동 107-6
...,...,...
1585,제주특별자치도 제주시 한림읍 한림상로 62,제주특별자치도 제주시 한림읍 동명리 1858
1586,제주특별자치도 제주시 도령로11길 26-9,제주특별자치도 제주시 연동 2313-4
1587,제주특별자치도 제주시 연북로 631,제주특별자치도 제주시 아라이동 3007-2
1588,제주특별자치도 서귀포시 남원읍 태위로663번길 1-1,제주특별자치도 서귀포시 남원읍 남원리 116-14


In [29]:
#final_fb.to_csv('PRE_F&B데이터.csv', encoding='utf-8-sig', index=False)

## 3. 가격정보 추가

- 가격 정보
- merchant data는 최신년도인 2022 기준 (2022없는 경우은 2021) 
- 연령은 20대만 필터링
- 요일시간대 승인건수 합으로 매출구간 중위값 나누어서 예상 가격대 계산


In [30]:
#final_fb = pd.read_csv('PRE_F&B데이터.csv')

### 3 - (1) MERCHANT 가격정보 전처리

In [31]:
merchant = pd.read_csv('data/JEJU_MERCHANT.csv', encoding='cp949')

In [32]:
# 요식/유흥 카테고리+ 연령 20대만 필터링
merchant = merchant[merchant.LG_CAT == 1]
merchant = merchant[(merchant.AGE_GP == "1_20_24") | (merchant.AGE_GP == "2_25_29")]

In [33]:
merchant['MNTH_SALES_PCTL'].unique()

array(['3_25%~50%', '4_50%~75%', '2_10%~25%', '5_75%~90%', '1_상위10%이상',
       '6_90%미만'], dtype=object)

In [34]:
# 매출분위 별 구간 중위값으로 대체
conditions = [
    merchant['MNTH_SALES_PCTL'] == "1_상위10%이상",
    merchant['MNTH_SALES_PCTL'] == "2_10%~25%",
    merchant['MNTH_SALES_PCTL'] == "3_25%~50%",
    merchant['MNTH_SALES_PCTL'] == "4_50%~75%",
    merchant['MNTH_SALES_PCTL'] == "5_75%~90%",
    merchant['MNTH_SALES_PCTL'] == "6_90%미만"
]

vals = [195750000,
        (195750000+11400000)/2,
        (114000000+63000000)/2,
        (63000000+28500000)/2,
        (28500000+ 10500000)/2,
        10500000]


merchant['SALES'] = np.select(conditions, vals)

In [35]:
vals

[195750000, 103575000.0, 88500000.0, 45750000.0, 19500000.0, 10500000]

In [36]:
# 2022 
merchant22 = merchant[merchant.YR==2022].groupby(['MCT_NM','MCT_BSE_AR','SALES']).apply(lambda x:  x.UE_CT.sum()).reset_index()
merchant22.columns = ['NCT_NM','MCT_BSE_AR', 'SALES', 'CT_SUM']

In [37]:
# 2021
merchant21 = merchant[merchant.YR==2021].groupby(['MCT_NM', 'MCT_BSE_AR', 'SALES']).apply(lambda x:  x.UE_CT.sum()).reset_index()
merchant21.columns = ['NCT_NM','MCT_BSE_AR', 'SALES','CT_SUM']  

In [38]:
name_list = merchant22['NCT_NM'].to_list()
index = []

for i in range(len(merchant21)):
    if merchant21['NCT_NM'][i] not in name_list:
        index.append(i)

In [39]:
#2022년 데이터가 없는 식당들은 2021년으로 대체
fin_merchant = pd.concat([merchant22, merchant21.loc[index,:]]).reset_index(drop=True)
fin_merchant.head()

,NCT_NM,MCT_BSE_AR,SALES,CT_SUM
0,(유)아웃백스테이크하우스코리아제주점,제주 제주시 노형동,45750000.0,966
1,(주) 베이힐,제주 서귀포시 하예동,195750000.0,672
2,(주) 신세계푸드 데블스도어 제주점,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38,88500000.0,1617
3,(주) 신세계푸드 제주신화월드 티플레이스,제주특별자치도 서귀포시 안덕면 녹차분재로 217,88500000.0,1218
4,(주)대유산업,제주 서귀포시 상예로 381,103575000.0,2114


### 3 - (2) 이름/주소 전처리

In [40]:
fin_merchant['NCT_NM_PRE'] = list(map(lambda x : name_preprocessing(x), fin_merchant['NCT_NM']))

In [41]:
fin_merchant['ADDR_PRE'] = list(map(lambda x: addr_preprocessing(x), fin_merchant['MCT_BSE_AR']))

### 3 - (3) 병합

In [42]:
final_fb2 = pd.merge(final_fb, fin_merchant, how='left', left_on=['AREA_NM_PRE', 'ROAD_ADDR'], right_on = ['NCT_NM_PRE','ADDR_PRE']).reset_index(drop=True)

In [43]:
print(len(final_fb2['SALES'].dropna())) # 82개만 가격정보 반영

82


## 4. 지번주소 기준 온실가스배출량 데이터 병합


- 지번주소별 탄소배출량 mean, max 데이터 병합 => 최종적으로 max 데이터사용 
- 공간 탄소배출량으로 사용


In [44]:
cd_dat = pd.read_csv('data/PRE_CO2_지번단위탄소배출량_meanmax.csv')

In [45]:
def lt_preprocess_add(add):
    add = add.strip()
    add = add.rstrip('번지')
    return add 

In [46]:
cd_dat['LTNO_ADDR'] = list(map(lambda x: lt_preprocess_add(x), cd_dat['LTNO_ADDR']))
cd_dat['LTNO_ADDR_PRE'] = list(map(lambda x: addr_preprocessing(x), cd_dat['LTNO_ADDR']))

In [47]:
fb_dat = pd.merge(left=final_fb2, right=cd_dat, how='left', left_on='LN_ADDR', right_on='LTNO_ADDR_PRE').reset_index(drop=True)

In [48]:
fb_dat = fb_dat.drop(['NCT_NM', 'MCT_BSE_AR', 'NCT_NM_PRE', 'ADDR_PRE', 'GPS_LNGTD', 'GPS_LTTD', 'LTNO_ADDR_PRE'], axis=1).reset_index(drop=True)

In [49]:
fb_dat.to_csv('F&B데이터_co2.csv', encoding='utf-8-sig', index=False) # 이 데이터 기준으로 네이버 검색 크롤링

## 5. 가격정보 대체

네이버 검색결과 크롤링 데이터와 기존 F&B 데이터 병합  

데이터 정제 및 가격 정보 NA를 네이버맵 메뉴정보 평균값으로 대체

### 5-(1) 데이터 병합

In [50]:
#fb_data = pd.read_csv('F&B데이터_탄소추가.csv')

In [51]:
naver_crawl = pd.read_csv('data/F&B데이터_네이버크롤링.csv')
naver_crawl.head()

,name_list,type_list,address_list,visitor_review_num_list,blog_review_num_list,visitor_review_list,menu_list,dl_mood_list,dl_topic_list,dl_why_list,dl_20_pop_list,org_list
0,1080칼국수,"칼국수,만두",제주 제주시 서광로13길 3,63,13,"{'음식이 맛있어요': 17, '친절해요': 12, '가성비가 좋아요': 11, '...",6750,NaN,NaN,NaN,NaN,1080칼국수
1,1950하우스,민박,제주 제주시 구좌읍 덕평로 364-2,0,1,NaN,알 수 없음,NaN,NaN,NaN,NaN,1950하우스
2,2093하우스,돈가스,제주 제주시 조천읍 남조로 2093,442,473,"{'음식이 맛있어요': 102, '주차하기 편해요': 48, '친절해요': 39, ...",14250,"아늑한, 분위기있는, 깨끗한, 엔틱한","전복죽, 흑돼지, 돈가스, 아이스크림","가족여행, 신선한, 데이트, 싱싱한",60.932499999999976%,2093하우스
3,24시누름돌김치찌개,한식,제주 제주시 노연로 146,426,42,"{'음식이 맛있어요': 128, '가성비가 좋아요': 70, '양이 많아요': 38...",7450,NaN,NaN,NaN,NaN,24시누름돌김치찌개
4,24시뼈다귀탕동홍점,24시뼈다귀탕,제주 서귀포시 동홍남로 43,284,19,"{'음식이 맛있어요': 90, '양이 많아요': 34, '매장이 넓어요': 19, ...",23000,NaN,NaN,NaN,NaN,24시뼈다귀탕동홍점


In [52]:
fb_dat = pd.merge(fb_dat, naver_crawl,how='left',
                  left_on='AREA_NM_PRE', right_on='org_list').reset_index(drop=True)

In [53]:
#fb_dat.to_csv('F&B데이터_탄소추가_크롤링추가.csv', encoding='utf-8-sig', index=False)

### 5-(2) 데이터 정제

In [54]:
#pd.set_option('display.max_columns', None) 
#pd.set_option('display.max_rows', None) 

#### 1) 검색 결과가 없는 데이터 삭제
- name_list가 no_result인 경우 네이버에 검색결과가 아예없는 것이므로 폐업으로 간주하고 전부 삭제



In [55]:
fb_dat = fb_dat[fb_dat.name_list != "no_result"].reset_index(drop=True)

- address_list가 NA인 경우(5가지 -> 제대로 검색되지 않음) 도 삭제

In [56]:
fb_dat = fb_dat.dropna(subset=['address_list']).reset_index(drop=True)

#### 2) 매칭이 제대로 안된 경우 확인

- 이름이 다른 경우 -> 주소 확인하고 주소 읍면동 다르면 삭제


In [57]:
def name_preprocessing(name):
    name = name.replace(' (', '(').replace(') ', ')').strip()
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    name = name.replace(" ", "")
    return name.strip()

In [58]:
def addr_preprocessing(addr):
    addr = addr.strip()
    if '제주특별자치도' in addr:
        addr = "제주특별자치도 " + addr.split('제주특별자치도')[1].lstrip()
    else:
        addr = re.sub('제주 서귀포시','제주특별자치도 서귀포시', addr)
        addr = re.sub('제주 제주시','제주특별자치도 제주시', addr)

    addr = re.sub(" ", "", addr) # 띄어쓰기제거
    addr = addr[7:14] # 7번째에서 14번째 글자만 보자 => 주소 같은곳인지 빠르게 확인하기 위함 
    return addr.strip() 

In [59]:
# 이름이 다른 경우만 추출
diff_df = fb_dat[fb_dat.name_list != fb_dat.org_list][['org_list', 'name_list', 'ROAD_ADDR', 'address_list', 'type_list']]

In [60]:
# 이름, 주소 비교가능하도록 정제
diff_df['name_list'] =list(map(lambda x : name_preprocessing(x), diff_df['name_list']))
diff_df['ROAD_ADDR'] = list(map(lambda x : addr_preprocessing(x), diff_df['ROAD_ADDR']))
diff_df['address_list'] =list(map(lambda x : addr_preprocessing(x), diff_df['address_list']))

In [61]:
#주소 앞 12글자가 다른 경우 삭제 
miss_index = diff_df[(diff_df.address_list != diff_df.ROAD_ADDR)].index

In [62]:
fb_dat = fb_dat.drop(miss_index).reset_index(drop=True)

#### 3) 음식점/카페가 아닌 경우 삭제
- type list 기준으로 식당 아닌 곳 제거


- 민박/관람,체험/화장실/농공시설/서점/기타숙박업/펜션/게스트하우스/패션/식물원,수목원/전통공예/세제제조/오름/원예,화훼농원/공방/4성급/마을회관/갤러리,화랑/뉴스,미디어/강,하천/테마공원/촬영장소/판촉,기념품/3성급/피부,체형관리/과일/슈퍼,마트/주류제조/식료품제조 => 삭제 


- 식료품/플라워카페/북카페/키즈카페,실내놀이터/방앗간 => 비슷한 다른 업종분류로 변경


In [63]:
# F&B데이터가 아닌 type 삭제
except_list = ['관람,체험', '민박', '화장실', '농공시설', '서점', '기타숙박업', '펜션', '게스트하우스', '패션',
              '식물원,수목원', '전통공예', '세제제조', '오름', '원예,화훼농원', '공방', '4성급', '마을회관',
              '갤러리,화랑', '뉴스,미디어', '강,하천', '테마공원', '촬영장소', '판촉,기념품', '3성급', '피부,체형관리', 
              '슈퍼,마트','주류제조','식료품제조', '과일', '회관']

fb_dat = fb_dat.loc[[i for i in range(len(fb_dat)) if fb_dat['type_list'][i] not in except_list]]

In [64]:
# 일품순두부 -> 두부요리로 편입
fb_dat['type_list'][fb_dat.type_list == "일품순두부"] = "두부요리"

In [65]:
fb_dat = fb_dat.reset_index(drop=True)
len(fb_dat)

1242

### 5-(3) 가격 데이터 정제





In [66]:
# 메뉴 평균가격(menu_list)이 알수없음인 경우 NA로 변경
fb_dat['menu_list'] = pd.to_numeric([np.NaN if menu == '알 수 없음' else menu for menu in fb_dat['menu_list']])

- merchant data 매출구간 중위값/20대 승인건수 합으로 계산한 예상가격대와 네이버 크롤링 가격대 비교

In [67]:
# 매출구간 중위값/20대고객 승인건수 합
fb_dat['merchant'] = fb_dat['SALES']/fb_dat['CT_SUM']

In [68]:
# merchant 데이터로 계산한 예상가격대와 네이버 크롤링 가격대 비교하기
# 두 데이터가 모두 존재하는 경우 차이가 얼마나 나는지 비교
diff = fb_dat[-fb_dat.merchant.isnull()][['name_list','merchant', 'menu_list']]
diff['diff'] = pd.to_numeric(diff['merchant'] - diff['menu_list'])

In [69]:
# 두 예상가격대 비교 
diff[-diff.menu_list.isnull()].describe()

,merchant,menu_list,diff
count,67.000000,67.000000,67.000000
mean,34513.590991,28726.611940,5786.979050
std,39203.024101,41265.207342,19768.948281
min,5889.459378,2600.000000,-65803.571429
25%,14427.906588,7500.000000,1549.252808
50%,23720.182257,13500.000000,6991.605955
75%,38939.048569,32625.000000,11564.992939
max,274007.936508,225000.000000,58275.635408


상대적으로 네이버크롤링 메뉴가격이 저평가되어있음   

예상 원인 : [칠돈가 표선직영점 네이버지도](https://map.naver.com/v5/search/%EC%B9%A0%EB%8F%88%EA%B0%80%20%ED%91%9C%EC%84%A0%EC%A7%81%EC%98%81%EC%A0%90/place/37182840?c=14118351.0001512,3938637.8930302,13,0,0,0,dh&isCorrectAnswer=true)
예시 처럼 김치전 추가시 1000원 데이터가 평균값을 확 내려버릴 수도 있고, 전체 메뉴가 아닌 일부 메뉴만 수집해오는 것이기 때문에 평균 매출가격을 대체한다고 보기 어려움 + MERCHANT 매출액 상위 가맹점 데이터이므로 고평가되어 있을 가능성 존재하지만
merchant데이터가 조금더 실제 결제금액과 더 유사하다고 볼 수 있음

#### **가격정보 최종로직!**
> (1) merchant 데이터 기준으로 사용  
> (2) merchant 가격 데이터 없는 경우 - naver크롤링 메뉴 평균 가격으로 나머지 채우기  
> (3) 그래도 na인 경우는 업종별 평균으로 합치기 - 업종분류가 자기자신밖에 없는 경우 다른 업종으로 편입

In [70]:
# 1. merchant 데이터로 가격대 채우기
fb_dat['price'] = fb_dat['merchant']

In [71]:
# 2. merchant 데이터 없는 경우 -> 네이버 크롤링 결과로 채우기
fb_dat['price'] = [ fb_dat['menu_list'][i] if np.isnan(fb_dat['merchant'][i]) else fb_dat['merchant'][i] for i in range(len(fb_dat))]

In [72]:
# 3. 두 가격정보 모두 없는 경우 업종 분류
fb_dat[fb_dat.price.isna()]['type_list'].unique()

array(['한식', '차', '생선회', '카페', '카페,디저트', '국밥', '종합분식', '김밥', '아귀찜,해물찜',
       '플라워카페', '민속주점', '일식당', '양식', '요리주점', '중식당', '해물,생선요리', '패밀리레스토랑',
       '이탈리아음식', '프랑스음식', '백반,가정식', '돈가스', '퓨전음식', '라이브카페', '베이커리',
       '차,커피', '돼지고기구이', '육류,고기요리', '떡볶이', '치킨,닭강정', '백숙,삼계탕', '한정식',
       '건강음료', '와플', '아이스크림', '와인', '맥주,호프', '떡,한과', '식료품', '향토음식', '막국수',
       '주류', '아시아음식', '뷔페', '전복요리', '브런치', '방앗간'], dtype=object)

merchant와 네이버 크롤링 결과 모두 가격정보를 채울 수 없는 경우 -> 업종별 평균으로 대체  
이때 마주친 문제점

> (1) type이 unique한데 NA인 경우 대체할 값이 없다 -> 비슷한 업종으로 편입   
> (2) type이 unique하지는 않지만 모두 NA이다 -> 비슷한 업종으로 모두 편입

(1) type이 unique한데 NA인 경우

In [73]:
# type이 unique한 경우
val_cnt = fb_dat['type_list'].value_counts().to_frame().reset_index()
uniq_list = val_cnt[val_cnt.type_list == 1]['index'].to_list()

In [74]:
# type이 unique한 애들 중 price가 NA인 경우 
fb_dat[(fb_dat['type_list'].isin(uniq_list)) & fb_dat['price'].isna()]['type_list'].unique()

array(['민속주점', '라이브카페', '건강음료', '와플', '와인', '식료품', '막국수', '주류', '방앗간'],
      dtype=object)

In [75]:
fb_dat[(fb_dat['type_list'].isin(uniq_list)) & fb_dat['price'].isna()][['name_list', 'type_list', 'tags', 'heart']]

,name_list,type_list,tags,heart
146,닐모리동동,민속주점,#오징어튀김먹물파스타 #한라산빙수 #우도땅콩팬케이크 #지름떡팬케이크 #향토음식 #음...,51
452,벨롱,라이브카페,#카페 #커피 #샌드위치 #음식 #디저트 #아메리카노 #카페라떼 #카페모카 #에이드...,3
774,올레산야초,건강음료,#휴식/힐링 #체험관광 #친구 #음식 #눈꽃빙수 #샐러드 #차,10
783,와팡 중문주상절리점,와플,#와플 #디저트 #한라봉 #음식 #빵 #주스,4
866,인디안 썸머 애월,와인,#애월 #펍 #카페 #음식,1
878,일품순두부탑동점,식료품,#탑동 #순두부요리 #전골요리 #음식 #식당 #음식 #식당,0
911,제주돔베막국수,막국수,#막국수 #돔베고기 #안심채움 #맛있는제주만들기 #음식 #식당,3
929,농업회사법인제주와이너리,주류,#술 #감귤주 #귤로만 #전통주 #제주감귤 #음식 #전통주 #제주감귤 #음식,5
1172,할머니떡집 방앗간,방앗간,#서귀포올레시장 #할머니떡집 #오메기떡 #제주떡맛집 #음식 #카페 #제주떡맛집 #음...,0


In [76]:
# 가격정보 가지는 type list 중 비슷한 항목으로 편입 
fb_dat['type_list'][146] = "양식"
fb_dat['type_list'][452] = "카페"
fb_dat['type_list'][774] = "카페,디저트"
fb_dat['type_list'][783] = "카페,디저트"
fb_dat['type_list'][866] = "바(BAR)"
fb_dat['type_list'][878] = "두부요리"
fb_dat['type_list'][911] = "국수"
fb_dat['type_list'][929] = "바(BAR)"
fb_dat['type_list'][1172] = "떡,한과"

(2) type이 unique하지는 않지만 모두 NA인 경우
- 차,커피 -> 카페
- 떡,한과 -> 떡카페

In [77]:
fb_dat['type_list'][491] = "카페"
fb_dat['type_list'][1141] = "카페"
fb_dat['type_list'][873] = "떡카페"
fb_dat['type_list'][907] = "떡카페"
fb_dat['type_list'][1172] = "떡카페"

### 5-(4) 가격 NA 평균 대체
업종구분(type_list) 평균으로 가격 NA값 대체

In [78]:
type_pr_mean = fb_dat[-fb_dat.price.isnull()]['price'].groupby(fb_dat['type_list']).mean().to_frame().reset_index()
type_pr_mean.sort_values(by=['price'], axis=0, ascending=False)

,type_list,price
40,생선회,86093.792128
44,"스테이크,립",65537.500000
93,해산물뷔페,63250.000000
42,소고기구이,62900.000000
51,오리요리,52500.000000
...,...,...
23,만두,5625.000000
10,기사식당,5250.000000
49,아이스크림,5200.000000
78,케이크전문,4500.000000


In [79]:
for i in fb_dat[fb_dat['price'].isna()].index:
    for j in range(len(type_pr_mean)):
        if fb_dat['type_list'][i] == type_pr_mean['type_list'][j]:
            fb_dat['price'][i] = type_pr_mean['price'][j]

## 6. 메뉴군 분류 및 평균 탄소 배출량 NA대체

가격과 마찬가지로 탄소배출량이 NA인 곳들도 업종별 평균으로 대체  
다만 탄소배출량의 경우 네이버 업종으로 na를 대체할 수 있는 데이터가 부족하므로
더 포괄적인 중분류 카테고리를 만들어서 채워줄 예정

In [80]:
# 필요없는 column 삭제
fb_dat.drop(['SALES', 'CT_SUM', 'LTNO_ADDR', 'ADDR','co2_mean', 'menu_list', 'merchant'], axis=1, inplace=True)

### 6-(1) 업종 중분류  카테고리 나누기

네이버 크롤링 데이터 기준 업종이 지나치게 세분화되어 있기 때문에 탄소배출량 업종별 평균으로 대체 불가하므로 보다 포괄적인 중분류 카테고리 생성
- 중분류 별 탄소배출량 평균으로 대체
> ex) 프랑스음식 식당인데 눈에 띄게 탄소배출량이 높은 경우 호텔과 같은 지번주소를 공유하고 있는 경우 존재 -> 양식이라는 큰 카테고리로 묶어준다면 더 많은 동종업과 묶이게 되므로 양식 종류의 음식점 전체에서 배출되는 탄소배출량의 평균으로 대체 가능

- 관련 카테고리를 위주로 묶되 평균 탄소배출량 값을 확인하고 눈에 띄는 값과 그에 합당한 이유가 있다고 판단되는 업종만 따로 구분함   
> ex) 24시해장국은 다른 국밥류 식당보다 탄소배출량 월등히 높음 -> 24시간 운영하는 식당이기 때문으로 판단되므로 따로 분류  
   
- 모든 경우를 예측할 수 없는 outlier는 포함하고 계산

In [81]:
fb_dat[fb_dat['type_list'] == "프랑스음식"][['AREA_NM_PRE', 'type_list',  'co2_max']].sort_values('co2_max', ascending=False)

,AREA_NM_PRE,type_list,co2_max
405,밀리우,프랑스음식,4.742963
675,아따블르,프랑스음식,NaN


In [82]:
# 프랜차이즈본사-> 돈가스로 변경
fb_dat[fb_dat['type_list'] == "프랜차이즈본사"]
fb_dat['type_list'][215] = "돈가스"

In [83]:
# 업종카테고리별 탄소배출량 평균 (NA없는 경우)
type_co2 = fb_dat[-fb_dat.co2_max.isna()].groupby('type_list').size().to_frame().reset_index()
type_co2['mean_co2'] = fb_dat[-fb_dat.co2_max.isna()].groupby('type_list').mean()['co2_max'].to_list()
type_co2.sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
32,뷔페,3,7.894728
75,프랑스음식,1,4.742963
38,소고기구이,5,0.923307
24,바(BAR),3,0.695014
22,"맥주,호프",9,0.683408
...,...,...,...
63,차,2,0.034635
17,떡볶이,4,0.028674
34,샌드위치,1,0.026467
43,"아귀찜,해물찜",1,0.024478


음식종류가 비슷하면 배출되는 탄소량도 비슷할 것이라는 가정 하에 비슷한 품목끼리 묶어서 중분류 카테고리 만들자   
 - 이때의 탄소배출량은 음식으로부터 예상되는 탄소배출량은 아니지만 비슷한 조리방법으로 인해 가스사용량 등 배출되는 탄소량이 비슷할 것으로 가정

In [84]:
fb_dat['category'] = np.nan

#### (1) 카페류 
- 카페/카페,디저트/플라워카페/북카페/떡카페/브런치/샌드위치/베이글/케이크전문/아이스크림/테이크아웃커피/차/과일,주스전문점 묶기
- 베이커리 & 테마카페 탄소배출량 outlier -> 따로 분류

In [85]:
temp = ['카페', '카페,디저트', '플라워카페', '북카페', '떡카페',
        '브런치', '샌드위치', '베이글', '케이크전문', '베이커리', '테마카페',
        '아이스크림','테이크아웃커피', '차', '과일,주스전문점', '와플']

type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
72,테마카페,1,0.606763
27,베이커리,32,0.543075
4,"과일,주스전문점",1,0.241005
69,"카페,디저트",107,0.172054
73,테이크아웃커피,2,0.162473
18,떡카페,2,0.133884
68,카페,45,0.128829
33,브런치,8,0.109503
44,아이스크림,3,0.073336
76,플라워카페,1,0.042569


In [86]:
# 베이커리/테마카페 분리
temp.remove('베이커리')
temp.remove('테마카페')

# 카페류 업종분류 -> 카페,디저트로 분류
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "카페/디저트"
fb_dat['category'][fb_dat['type_list'] == "베이커리"] = "베이커리"
fb_dat['category'][fb_dat['type_list'] == "테마카페"] = "테마카페"

#### (2) 육류

- 육류고기요리/족발보쌈/곱창막창양/정육식당/닭발/닭요리/돈가스/치킨닭강정/스테이크립/닭갈비/샤브샤브
- 소고기구이/돼지고기구이 따로 분류

In [87]:
temp = ['돼지고기구이', '육류,고기요리', '족발,보쌈', '곱창,막창,양','정육식당','소고기구이','오리요리',
        '닭발', '닭요리', '돈가스', '치킨닭강정', '스테이크,립','닭갈비', '샤브샤브', '치킨,닭강정']

type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
38,소고기구이,5,0.923307
14,돼지고기구이,50,0.337410
11,닭갈비,1,0.250594
50,"육류,고기요리",22,0.193225
12,닭요리,3,0.184112
13,돈가스,8,0.175953
3,"곱창,막창,양",1,0.175668
57,정육식당,2,0.147335
46,오리요리,1,0.121349
66,"치킨,닭강정",7,0.111913


In [88]:
# 소고기구이/돼지고기구이 분리
temp.remove('소고기구이')
temp.remove('돼지고기구이')

# 육류 업종분류 -> 육류고기요리로 분류
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "육류/고기"
fb_dat['category'][fb_dat['type_list'] == "소고기구이"] = "소고기구이"
fb_dat['category'][fb_dat['type_list'] == "돼지고기구이"] = "돼지고기구이"

#### (3) 한정식
- 한식/한정식/백반가정식/쌈밥/기사식당/두부요리/죽

In [89]:
temp = ['한식', '한정식', '백반,가정식', '쌈밥', '비빔밥', '기사식당', '두부요리', '죽']
type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
8,기사식당,1,0.186997
15,두부요리,4,0.178896
78,한식,124,0.159088
42,쌈밥,2,0.143768
80,한정식,9,0.131949
25,"백반,가정식",6,0.112664
60,죽,1,0.077957


In [90]:
# 한식(한정식) 업종분류 -> 한정식으로 분류
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "한식/한정식"

#### (4) 탕/국밥/전골류
- 곰탕,설렁탕/추어탕/국밥/백숙,삼계탕/갈비탕/감자탕/해장국/순대,순댓국
- 24시뼈다귀탕 따로 

In [91]:
temp = ['곰탕,설렁탕', '찌개,전골', '추어탕', '국밥', '백숙,삼계탕','매운탕,해물탕',
        '24시뼈다귀탕', '갈비탕', '감자탕', '해장국', '순대,순댓국']
type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
0,24시뼈다귀탕,1,0.402918
39,"순대,순댓국",4,0.296415
2,"곰탕,설렁탕",3,0.274153
26,"백숙,삼계탕",1,0.241005
21,"매운탕,해물탕",2,0.206992
1,갈비탕,1,0.186997
62,"찌개,전골",3,0.122013
82,해장국,10,0.118895
5,국밥,2,0.112024
65,추어탕,2,0.076846


In [92]:
# 24시뼈다귀탕 -> 24시국밥
temp.remove('24시뼈다귀탕')

# 탕/국밥/전골 업종분류 -> 탕류로 분류
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "국밥/탕"
fb_dat['category'][fb_dat['type_list'] == "24시뼈다귀탕"] = '24시국밥/탕'

#### (5) 해물/생선류 

- 해물,생선요리/생선회/전복요리/생선구이/굴요리/장어,먹장어요리/낙지요리/조개요리/복어요리/오징어요리/게요리/아귀찜,해물찜 

In [93]:
temp = ['해물,생선요리', '생선회', '전복요리', '생선구이', '굴요리','장어,먹장어요리', '해산물뷔페',
        '낙지요리', '조개요리', '복어요리', '오징어요리', '게요리', '아귀찜,해물찜']
type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
55,"장어,먹장어요리",1,0.264706
29,복어요리,1,0.247921
35,생선구이,2,0.222246
10,낙지요리,1,0.220163
36,생선회,39,0.143597
56,전복요리,10,0.127678
81,"해물,생선요리",38,0.116671
58,조개요리,1,0.103958
7,굴요리,1,0.096032
47,오징어요리,1,0.057620


In [94]:
# 해산물/생선요리 업종분류 -> 해산물/생선로 분류
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "해산물/생선"

#### (6) 주류/술집
- 맥주,호프 카테고리의 탭하우스더코너가 호텔1층에 있어서 지번주소 탄소배출량 매우 크게 나타남
- 하지만 나머지 애들은 탄소배출량 비슷.. 하나의 outlier는 그대로 두고 평균 계산하기로 결정
- 맥주,호프/바(BAR)/이자카야/포장마차

In [95]:
temp = ['맥주,호프', '바(BAR)', '이자카야', '포장마차', '요리주점']
type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
24,바(BAR),3,0.695014
22,"맥주,호프",9,0.683408
51,이자카야,2,0.160577
48,요리주점,2,0.046143


In [96]:
# 맥주호프/바/이자카야/포장마차 업종분류 -> 술집/주류로 분류
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "술집/주류"

#### (7) 외국음식

- 중식당/딤섬,중식만두/양식/이탈리아음식/스파게티파스타전문/일식당/초밥,롤/일본식라면/우동,소바/멕시코,남미음식/퓨전음식/카레/인도음식/태국음식/아시아음식/양식
- 패밀리레스토랑만 따로 계산

In [97]:
temp = ['중식당', '딤섬,중식만두', '양식', '패밀리레스토랑', '이탈리아음식','스파게티,파스타전문',
       '일식당', '초밥,롤', '일본식라면', '우동,소바', '멕시코,남미음식', '퓨전음식',
        '카레', '인도음식', '태국음식', '아시아음식', '베트남음식', '프랑스음식']
type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
75,프랑스음식,1,4.742963
74,패밀리레스토랑,9,0.454112
45,양식,21,0.369495
16,"딤섬,중식만두",1,0.240337
54,일식당,8,0.209299
61,중식당,26,0.200496
49,"우동,소바",1,0.188241
23,"멕시코,남미음식",1,0.143450
64,"초밥,롤",3,0.124245
41,"스파게티,파스타전문",1,0.123323


In [98]:
temp.remove('패밀리레스토랑')

# 외국음식(양식, 아시아음식, 남미음식, 퓨전음식) 업종분류 -> 외국음식으로 분류
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "외국음식"
fb_dat['category'][fb_dat['type_list'] == "패밀리레스토랑"] = '패밀리레스토랑'

#### (8) 분식 

- 칼국수,만두/김밥/국수/종합분식/분식/만두/라면/떡볶이

In [99]:
temp = ['칼국수,만두', '종합분식', '국수', '라면', '김밥', 
        '떡볶이', '도시락,컵밥', '만두', '분식']

type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
9,김밥,9,0.209775
70,"칼국수,만두",11,0.205019
6,국수,24,0.127283
59,종합분식,15,0.120456
31,분식,1,0.109538
20,만두,1,0.089924
19,라면,4,0.049648
17,떡볶이,4,0.028674


In [100]:
fb_dat['category'][fb_dat['type_list'].isin(temp)] = "분식"

#### (9) 기타

In [101]:
temp = ['햄버거', '뷔페', '향토음식', '피자', '한식뷔페']
type_co2[type_co2.type_list.isin(temp)].sort_values(by=['mean_co2'], axis=0, ascending=False)

,type_list,0,mean_co2
32,뷔페,3,7.894728
83,햄버거,6,0.428994
84,향토음식,19,0.154380
77,피자,4,0.118000
79,한식뷔페,1,0.070730


In [102]:
fb_dat['category'][fb_dat['type_list'].isin(['햄버거', '피자'])] = "패스트푸드"
fb_dat['category'][fb_dat['type_list'].isin(['향토음식'])] = "향토음식"
fb_dat['category'][fb_dat['type_list'].isin(['뷔페', '한식뷔페'])] = "뷔페"

In [103]:
fb_dat['category'].value_counts()

카페/디저트     310
한식/한정식     218
해산물/생선     151
외국음식       136
분식         100
육류/고기       73
돼지고기구이      62
베이커리        55
국밥/탕        43
술집/주류       23
향토음식        23
패스트푸드       19
패밀리레스토랑     11
뷔페           8
소고기구이        5
테마카페         3
24시국밥/탕      2
Name: category, dtype: int64

### 6-(2) 중분류 평균으로 탄소배출 NA 대체

In [104]:
# 중분류 카테고리 평균 탄소배출량 계산
cat_co2_mean = fb_dat[-fb_dat.co2_max.isnull()]['co2_max'].groupby(fb_dat['category']).mean().to_frame().reset_index()
cat_co2_mean.sort_values(by=['co2_max'], axis=0, ascending=False)

,category,co2_max
5,뷔페,5.938728
6,소고기구이,0.923307
11,테마카페,0.606763
3,베이커리,0.543075
7,술집/주류,0.540572
12,패밀리레스토랑,0.454112
0,24시국밥/탕,0.402918
2,돼지고기구이,0.337410
13,패스트푸드,0.304597
8,외국음식,0.289272


In [105]:
for i in fb_dat[fb_dat['co2_max'].isna()].index:
    for j in range(len(cat_co2_mean)):
        if fb_dat['category'][i] == cat_co2_mean['category'][j]:
            fb_dat['co2_max'][i] = cat_co2_mean['co2_max'][j]

In [106]:
#fb_dat.to_csv('F&B데이터_final.csv', encoding='utf-8-sig', index=False)

### 6-(3) 음식별 탄소배출량 데이터셋 만들기

참고할 수 있는 메뉴별 식재료별 탄소배출량 데이터 구하기 어려움  
모든 메뉴와 식재료별 탄소배출량 데이터가 모두 있는 것이 아니며, 식당별로 어떤 메뉴를 파는지 전부 파악 불가

=> 중분류 카테고리의 대표 메뉴별 (구할 수 있는) 탄소배출량 데이터를 통해 대체하고자 함! 
여러개의 업종을 하나의 중분류로 묶고, 중분류별 대표 메뉴를 선정하는데는 분석자의 주관이 많이 들어갈 수 밖에 없지만 참고,사용 가능한 데이터가 매우 부족한 상태에서 최대한 합리적으로 음식별 탄소배출량이라는 필요한 정보를 만들기 위한 고민 끝에 음식별 탄소배출량 변수 생성  
(향후 식재료 혹은 음식 별 탄소배출량을 계산하는 DB가 구축되고 활성화된다면 더욱 정교한 탄소배출량 계산이 가능할 것으로 예상)

In [107]:
#fb_data = pd.read_csv("F&B데이터_final.csv")

In [108]:
category_list = fb_dat['category'].unique()

<한끼밥상 탄소계산기 1인분 기준>

**볼드처리된 부분은 한끼밥상 내 음식으로 대체 불가한 메뉴의 평균 탄소배출량을 찾아 1인분 단위로 계산**  
    -> FOOD ITEM 데이터를 기준으로 1파운드를 kg단위로 환산 후 1인분으로 계산
    -> https://www.healabel.com/pasta-benefits-side-effects/ 에서 파스타, 피자, 카레, 초밥의 1인분 탄소배출량 데이터 확보
1. 소고기구이 :  7.7kgCO2e
2. 돼지고기구이 : 2kgCO2e
3. 육류/고기 : 닭볶음 1.6kgCO2e 
4. 국밥/탕, 24시국밥/탕 : 갈비탕 5kgCO2e, 설렁탕 10kgCO2e, 곰탕 9.7kgCO2e
5. 한식/한정식 : 밥 0.5kgCO2e, 불고기비빔밥 1.4kgCO2e 된장국 0.9kgCO2e 소고기뭇국 7.7kgCO2e 김치찌개 2.3kgCO2e 북엇국 1.1kgCO2e
6. 향토음식 : 해물칼국수 0.4kgCO2e, 국수장국 1.8kgCO2e 고등어구이 0.3kgCO2e 비빔국수 1.3kgCO2e 
7. 분식 : 김밥 0.4kgCO2e 해물칼국수 0.4kgCO2e 가래떡 0.1kgCO2e 어묵 0 감자전 0.4 비빔냉면 1.1 
8. 해산물/생선 : 고등어구이 0.3kgCO2e 생태찌개 2.9kgCO2e 생선회 0kgCO2e (해초무침이 0 인 것 고려하여 생선회도 0으로 예측) *생선은 탄소배출량 적다!*
9. 카페/디저트, 베이커리, 테마카페 : 에스프레스 0.3kgCO2e 라떼 0.6kgCO2e + **빵의 평균 탄소배출량(빵 0.073/1파운드, 케이크 0.564/1파운드, 패스츄리 0.874/1파운드)** *(빵은 100g당 1인분으로 계산 모닝빵 기준)*
10. 패스트푸드 : 피자 2kgCO2e, 햄버거 3.7kgCO2e 치킨 2.1kgCO2e 
11. 술집주류 : 한정식/향토음식/분식/해산물,생선의 평균 + **술의 평균 탄소배출량(0.45/1파운드)** *(맥주기준 500g(500cc) 1인분으로 계산)*
12. 패밀리 레스토랑, 외국음식 : **파스타 1.24kgCO2e, 카레 1.6kgCO2e, 피자 2kgCO2e, 초밥 1.36kgCO2e** 
13. 뷔페 - 한정식,분식,외국음식의 평균



In [109]:
# 빵 1kg당 평균 co2 
(0.074+0.564+0.874)/3/0.453

1.1125827814569536

In [110]:
# 맥주 1kg당 평균 co2
0.45/0.453

0.9933774834437086

In [111]:
# 카테고리별 탄소배출량 계산
food_co2_dict = {'소고기구이': 7.7,
                 '돼지고기구이':2,
                 '육류/고기':1.6,
                 '국밥/탕': np.mean([5, 10, 9.7]),
                 '24시국밥/탕':np.mean([5, 10, 9,7]),
                 '한식/한정식': np.mean([0.5, 1.4, 0.9, 7.7, 2.3, 1.1]),
                 '향토음식': np.mean([0.4, 1.8, 0.3, 1.3]),
                 '분식' : np.mean([0.4, 0.4, 0.1, 0.4, 1.1]),
                 '해산물/생선' : np.mean([0.3, 2.9, 0]),
                 '테마카페' : np.mean([0.3, 0.6, 1.112]),
                 '카페/디저트' : np.mean([0.3, 0.6, 1.1118/10]), # 빵 100g기준
                 '베이커리': np.mean([0.3, 0.6, 1.112]),
                 '패스트푸드': np.mean([2, 3.7, 2.1]),
                 '술집/주류': np.mean([np.mean([0.5, 1.4, 0.9, 7.7, 2.3, 1.1]),
                                  np.mean([0.4, 1.8, 0.3, 1.3]),
                                  np.mean([0.4, 0.4, 0.1, 0, 0.4, 0.3]),
                                  np.mean([0.3, 2.9, 0])]) + 0.9934/2,  # 맥주 500l기준
                 '패밀리레스토랑': np.mean([1.24, 1.6, 2, 1.36]),
                 '외국음식' : np.mean([1.24, 1.6, 2, 1.36]),
                 '뷔페' : np.mean([np.mean([0.5, 1.4, 0.9, 7.7, 2.3, 1.1]),np.mean([0.4, 0.4, 0.1, 0, 0.4, 0.3]),
                               np.mean([1.24, 1.6, 2, 1.36])])
                }

In [112]:
# 음식별 탄소배출량 변수 생성
fb_dat['food_co2'] = ''
fb_dat['food_co2'] = [food_co2_dict[cat] for cat in fb_dat['category']]

In [113]:
fb_dat['food_co2'].groupby(fb_dat['category']).mean().to_frame().reset_index().sort_values('food_co2', ascending=False)

,category,food_co2
1,국밥/탕,8.233333
0,24시국밥/탕,7.750000
6,소고기구이,7.700000
13,패스트푸드,2.600000
14,한식/한정식,2.316667
2,돼지고기구이,2.000000
7,술집/주류,1.646700
9,육류/고기,1.600000
8,외국음식,1.550000
12,패밀리레스토랑,1.550000


In [114]:
fb_dat.to_csv('data/F&B데이터_co2_final.csv', encoding='utf-8-sig', index=False)